In [2]:
import json
import random
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from bs4 import BeautifulSoup

OUT_FILE = "OUTPUT.json"

def into_json(org_id, name, address, benzin, prices):
    """ Шаблон файла OUTPUT.json"""
    return {
        "ID": org_id,
        "name": name,
        "address": address,
        "benzin": benzin,
        "prices": prices
    }

class JSONWorker:
    """ Класс для работы с JSON файлом"""
    @staticmethod
    def save_to_json(data):
        """ Сохранение данных в JSON файл """
        with open(OUT_FILE, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)

class InfoGetter:
    """ Класс с логикой парсинга данных из объекта BeautifulSoup"""
    @staticmethod
    def get_name(soup_content):
        """ Получение названия организации """
        try:
            name_element = soup_content.find("h1", {"class": "orgpage-header-view__header"})
            if name_element:
                return name_element.getText()  # Получаем текст из элемента
            else:
                print("Элемент с названием не найден.")
                return ""
        except Exception as e:
            print(f"Ошибка при получении названия: {e}")
            return ""

    @staticmethod
    def get_address(soup_content):
        """ Получение адреса организации """
        try:
            address_element = soup_content.find("a", {"class": "business-contacts-view__address-link"})
            if address_element:
                return address_element.getText()  # Получаем текст из элемента
            else:
                print("Элемент с адресом не найден.")
                return ""
        except Exception as e:
            print(f"Ошибка при получении адреса: {e}")
            return ""

    @staticmethod
    def get_benzin(soup_content):
        """ Получение списка топлива и цен"""
        benzin = []
        prices = []
        try:
            # Получаем список топлива
            fuel_elements = soup_content.find_all("div", {"class": "search-fuel-info-view__name"})
            for benzin_s in fuel_elements:
                benzin.append(benzin_s.getText().strip())  # Убираем лишние пробелы

            # Получаем список цен
            price_elements = soup_content.find_all("div", {"class": "search-fuel-info-view__value"})
            for price_s in price_elements:
                prices.append(price_s.getText().strip())  # Убираем лишние пробелы

            # Проверяем, что количество видов топлива соответствует количеству цен
            if len(benzin) != len(prices):
                print("Количество видов топлива не соответствует количеству цен.")

        except Exception as e:
            print(f"Ошибка при получении топлива и цен: {e}")
        
        return benzin, prices
import polars as pl

class GrabberApp:
    def __init__(self):
        self.data = []
        self.processed_urls = set()  # Множество для отслеживания обработанных URL

    def grab_data(self):
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)
        driver.maximize_window()
        driver.get('https://yandex.ru/maps/213/moscow/search/%D0%B7%D0%B0%D0%BF%D1%80%D0%B0%D0%B2%D0%BA%D0%B8/')

        # Прокрутка страницы вниз для загрузки всех результатов
        for _ in range(3):  # Прокручиваем 3 раза
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            sleep(2)  # Ждем, чтобы страница загрузилась

        try:
            while True:
                # Ожидаем, пока элементы загрузятся
                organizations_list = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CLASS_NAME, 'search-list-view__list'))
                )
                
                # Получаем все элементы списка
                org_elements = organizations_list.find_elements(By.TAG_NAME, 'li')

                # Фильтруем элементы, которые уже были обработаны
                new_org_elements = [org for org in org_elements if org.find_element(By.TAG_NAME, 'a').get_attribute("href") not in self.processed_urls]

                if not new_org_elements:
                    print("Нет новых организаций для обработки.")
                    break  # Выходим из цикла, если больше нет новых организаций

                for org_element in new_org_elements:
                    # Получаем ссылку на организацию
                    try:
                        organization_url = org_element.find_element(By.TAG_NAME, 'a').get_attribute("href")
                    except StaleElementReferenceException:
                        print("Ссылка на элемент устарела, пропускаем.")
                        continue  # Пропускаем текущую итерацию и продолжаем цикл

                    print(f"Обрабатываем организацию: {organization_url}")

                    # Переходим по URL организации
                    driver.get(organization_url)
                    sleep(random.uniform(2, 5))  # Ждем, чтобы страница загрузилась

                    # Получаем данные с помощью BeautifulSoup
                    soup = BeautifulSoup(driver.page_source, "lxml")  # Здесь добавляем BeautifulSoup
                    name = InfoGetter.get_name(soup)  # Здесь получаем название
                    print(name)
                    address = InfoGetter.get_address(soup)  # Получаем адрес
                    print(address)
                    benzin, prices = InfoGetter.get_benzin(soup)  # Получаем топливо и цены
                    print(benzin, prices)

                    # Формируем данные в формате словаря
                    output = {
                        "url": organization_url,
                        "name": name,
                        "address": address,
                        "benzin": benzin,
                        "prices": prices
                    }
                    self.data.append(output)  # Сохраняем данные в список

                    print(f'Данные добавлены для {organization_url}')
                    self.processed_urls.add(organization_url)  # Добавляем URL в множество обработанных

                    # Возвращаемся к списку организаций
                    driver.back()
                    sleep(random.uniform(2, 5))  # Ждем, чтобы страница загрузилась

        except Exception as e:
            print(f"Ошибка: {e}")

        JSONWorker.save_to_json(self.data)
        print('Данные сохранены в OUTPUT.json')
        # Сохраняем все собранные данные в DataFrame Polars
        df = pl.DataFrame(self.data)
        print(df)  # Выводим DataFrame на экран

        # Сохраняем DataFrame в файл
        driver.quit()

def main():
    grabber = GrabberApp()
    grabber.grab_data()

if __name__ == '__main__':
    main()


Обрабатываем организацию: https://yandex.ru/maps/org/lukoyl/1131889882/
Лукойл
Олимпийский просп., 5, стр. 1, Москва
['АИ 92+', 'АИ 95+', 'АИ-98', 'ДТ+', 'АИ 100'] ['57,63', '65,60', '–', '70,70', '89,36']
Данные добавлены для https://yandex.ru/maps/org/lukoyl/1131889882/
Ссылка на элемент устарела, пропускаем.
Ссылка на элемент устарела, пропускаем.
Ссылка на элемент устарела, пропускаем.
Ссылка на элемент устарела, пропускаем.
Ссылка на элемент устарела, пропускаем.
Ссылка на элемент устарела, пропускаем.
Обрабатываем организацию: https://yandex.ru/maps/org/rosneft/148354456029/
Роснефть
Котельническая наб., 1/15соор10, Москва
['АИ-92', 'АИ-95', 'АИ-98', 'ДТ'] ['57,05', '62,95', '–', '–']
Данные добавлены для https://yandex.ru/maps/org/rosneft/148354456029/
Ссылка на элемент устарела, пропускаем.
Ссылка на элемент устарела, пропускаем.
Ссылка на элемент устарела, пропускаем.
Ссылка на элемент устарела, пропускаем.
Ссылка на элемент устарела, пропускаем.
Обрабатываем организацию: http